In [110]:
!pip install pyenchant
import nltk
nltk.download("words")
nltk.download("stopwords")
nltk.download("wordnet")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
!sudo apt-get install libenchant1c2a

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libenchant1c2a is already the newest version (1.6.0-11.3build1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [106]:
import pandas as pd
import numpy as np
import enchant
import string
import time
import regex
import re
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words as eng_words
from nltk.corpus import stopwords

In [156]:
class TextCleaning:
   def __init__(self,stemming: bool):
       self._ps = PorterStemmer()
       if type(stemming) is not bool:
        raise TypeError("Parameter < stemming > must be < bool >")
       else:
        self._stem = stemming
       self._eng = enchant.Dict("en_US")
       self._sw = self._imp_stop_words()
       self._double = np.squeeze(pd.read_csv(
           "https://raw.githubusercontent.com/NITHISHM2410/Text_Preprocessing/NLP/Text%20Cleaning/double_identical.txt",
           header=None).values).tolist()
       self._notwords = np.squeeze(pd.read_csv("https://raw.githubusercontent.com/NITHISHM2410/Text_Preprocessing/NLP/Text%20Cleaning/negative%20words.txt",
                                               header=None).values.tolist())
       self._notwords = self.ready_not_words()
       self._eng_words = self.ready_eng_words()
    

   def ready_eng_words(self):
       self._eng_words = eng_words.words()
       self._eng_words += self._notwords
       return self._eng_words

   def ready_not_words(self):
       self._notwords = [i.replace("'", "") for i in self._notwords]
       self._notwords.append("not")
       return self._notwords

   @staticmethod
   def _imp_stop_words():
       words = stopwords.words("english")
       words.append("hello")
       ntwords = [i for i in words if "n't" in i or i[-1] == 'n']
       ntwords.remove('between')
       ntwords.remove('again')
       ntwords.remove('on')
       ntwords.remove('an')
       ntwords.remove('won')
       ntwords.remove('when')
       ntwords.remove('than')
       ntwords.remove('then')
       ntwords = ntwords[5:]
       stop_words = set(words) - set(ntwords)
       stop_words.remove("not")
       return stop_words

   def not_list(self):
       return self._notwords

   def _duplicates(self,text):
       output = ""
       prev = ""
       for t in text:
           if len(output) == 0:
               output += t
               prev = t
           if t == prev:
               continue
           else:
               output += t
               prev = t
       return output

   def clean(self,sent):
       sent = sent.lower()
       sent = re.sub(r'[^\w\s]', '', sent)
       sent = re.sub(r'\d+', '', sent)
       sent = sent.split()

       for i in sent:
           if 'http' in i or 'html' in i or 'www' in i or '.com' in i:
               sent.remove(i)
       sent = [i for i in sent if not len(i) <= 2]

       for i in sent:
           ind = sent.index(i)
           if i in self._double:
               continue
           else:
               i = self._duplicates(i)
               sent[ind] = i  
       
       sent = [i for i in sent if i not in self._sw]

       if self._stem:
           sent = [self._ps.stem(i) for i in sent]
       sent = " ".join(sent)
       
       return sent

   

CREATING TEXT CLEANING INSTANCE

In [157]:
cleaner = TextCleaning(False)

EXAMPLES

In [158]:
cleaner.clean("food here was awesomeeeee hurrayyyyyyyyyy !!!!!!!!!!")

'food awesome huray'

In [159]:
cleaner.clean("WORST food experience......")

'worst food experience'

TRYING IT ON A REAL DATASET

In [162]:
dff = pd.read_csv("https://raw.githubusercontent.com/sharmaroshan/Restaurant-Reviews-Analysis/master/Restaurant_Reviews.tsv",sep = '\t')

In [163]:
dff

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [164]:
dff['Review_clean'] = dff['Review'].apply(cleaner.clean)

In [165]:
dff[['Review','Review_clean']]

,Review,Review_clean
0,Wow... Loved this place.,wow loved place
1,Crust is not good.,crust not good
2,Not tasty and the texture was just nasty.,not tasty texture nasty
3,Stopped by during the late May bank holiday of...,stoped late may bank holiday rick steve recome...
4,The selection on the menu was great and so wer...,selection menu great prices
...,...,...
995,I think food should have flavor and texture an...,think food flavor texture lacking
996,Appetite instantly gone.,apetite instantly gone
997,Overall I was not impressed and would not go b...,overal not impressed would not back
998,"The whole experience was underwhelming, and I ...",whole experience underwhelming think well ninj...
